### Only Try for Baseline Percentile Model

* scenario of what if no historical data.
* Also, to tune model data wise,   
* i.e. remove deregistered students at week 18, 
* rather than week 16 cos no longer has dependency on historical data
* Then do week 23, then week 30 

In [2]:
import numpy as np
import pandas as pd
import os
import math
print (pd.__version__)
print (np.__version__)
print (os.name)
print(hasattr(math, "isfinite"))
os.getcwd()
os.chdir('C:\\Users\\EeLian\\Desktop\\OULAD_KAGGLE\\PythonCodeCleanUp\\Data')

1.2.4
1.20.1
nt
True


In [3]:
# read datafile
studentInfo = pd.read_csv('studentInfo.csv')
courses = pd.read_csv('courses.csv')
studentRegistration = pd.read_csv('studentRegistration.csv')

# add _fl_latest_module_attended
studentInfo
studentInfo.sort_values(by= ['id_student','code_module','code_presentation'], inplace = True, ascending = False)
studentInfo['_fl_latest_module_attended'] = studentInfo.groupby(['id_student','code_module'])['id_student'].rank(method='first', ascending = False)
# for checking purposes
# studentInfo[studentInfo['id_student'].isin([584077])]

# merge/ left join to get studentInfo + courses + registration 
studentInfo2 = studentInfo.merge(courses, how='left', left_on=["code_module", "code_presentation"], right_on=["code_module", "code_presentation"])
studentInfo3 = studentInfo2.merge(studentRegistration, how='left', left_on=["code_module", "code_presentation","id_student"], right_on=["code_module", "code_presentation","id_student"])

# identifying actual dropout. dropout after semester starts=actual dropout; else, it's deregistration
studentInfo3.loc[studentInfo3['date_unregistration'] >0, '_flag_actual_dropout'] = 1
studentInfo3.loc[studentInfo3['date_unregistration'] <=0, '_flag_actual_dropout'] = 0
studentInfo3.loc[studentInfo3['date_unregistration'].isnull(), '_flag_actual_dropout'] = 0

In [4]:
# import data
studentAssessment = pd.read_csv('studentAssessment.csv')
assessments = pd.read_csv('assessments.csv')

# merge/ left join
# name as studentAssessment2
studentAssessment2 = studentAssessment.merge(assessments, how='left', left_on=["id_assessment"], right_on=["id_assessment"])

# add _flag_null_score just to check if there is submission
studentAssessment2['_flag_null_score']=studentAssessment2['score'].isnull().astype(int)

# add weighted score
studentAssessment2['_weighted_score'] = studentAssessment2['score']*studentAssessment2['weight']

# add pass fail. the NaN means didnt submit
studentAssessment2.loc[studentAssessment2['score'] >=40, '_flag_assessment_passed'] = 1
studentAssessment2.loc[studentAssessment2['score'] <40, '_flag_assessment_passed'] = 0
studentAssessment2["_date_group"] = pd.Series([], dtype=float)
studentAssessment2["_date_group"] = studentAssessment2['date_submitted']
studentAssessment2.loc[studentAssessment2['date_submitted'] <0, '_date_group'] = studentAssessment2['_date_group'].apply(lambda x: math.ceil(float(x/7)))
studentAssessment2.loc[studentAssessment2['date_submitted'] >=0, '_date_group'] = studentAssessment2['_date_group'].apply(lambda x: math.floor(float((x/7)+1)))
#studentAssessment2.head()

# for EEE Course, Cut off filter at Week 18
studentAssessment2_09 = studentAssessment2[(studentAssessment2['_date_group']<=9)]
studentAssessment2_18 = studentAssessment2[(studentAssessment2['_date_group']<=18)]
studentAssessment2_23 = studentAssessment2[(studentAssessment2['_date_group']<=23)]
studentAssessment2_30 = studentAssessment2[(studentAssessment2['_date_group']<=30)]

In [5]:
# sum up
tbl_total_weighted_score_09=studentAssessment2_09.\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'_weighted_score':['sum']})
tbl_total_weighted_score_09.columns = ['sum_weighted_score']
tbl_total_weighted_score_09 = tbl_total_weighted_score_09.reset_index()

tbl_total_weighted_score_18=studentAssessment2_18.\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'_weighted_score':['sum']})
tbl_total_weighted_score_18.columns = ['sum_weighted_score']
tbl_total_weighted_score_18 = tbl_total_weighted_score_18.reset_index()

tbl_total_weighted_score_23=studentAssessment2_18.\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'_weighted_score':['sum']})
tbl_total_weighted_score_23.columns = ['sum_weighted_score']
tbl_total_weighted_score_23 = tbl_total_weighted_score_23.reset_index()

tbl_total_weighted_score_30=studentAssessment2_18.\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'_weighted_score':['sum']})
tbl_total_weighted_score_30.columns = ['sum_weighted_score']
tbl_total_weighted_score_30 = tbl_total_weighted_score_30.reset_index()
##################################

# create distinct list
studentAssessment2_distinct=studentAssessment2[['id_student','code_module','code_presentation']]
studentAssessment2_distinct=studentAssessment2_distinct.drop_duplicates()

##################################
# merge
studentAssessment2_agg1_09 = studentAssessment2_distinct \
    .merge(tbl_total_weighted_score_09, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"])

studentAssessment2_agg1_18 = studentAssessment2_distinct \
    .merge(tbl_total_weighted_score_18, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"])

studentAssessment2_agg1_23 = studentAssessment2_distinct \
    .merge(tbl_total_weighted_score_23, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"])

studentAssessment2_agg1_30 = studentAssessment2_distinct \
    .merge(tbl_total_weighted_score_30, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"])

In [6]:
# import data
studentVle = pd.read_csv('studentVle.csv')
studentVle2 = studentVle
vle = pd.read_csv('vle.csv')

# merge data
studentVle3 = studentVle2.merge(vle, how='left', \
                                                 left_on=["code_module", "code_presentation","id_site"], \
                                                 right_on=["code_module", "code_presentation","id_site"])
#studentVle3.to_csv('studentVle3.csv',index=True,header=True)

# add date_group/ weeks
studentVle3['_date_group'] = pd.Series([], dtype=float)
studentVle3['_date_group'] = studentVle3['date']
studentVle3.loc[studentVle3['date'] <0, '_date_group'] = studentVle3['_date_group'].apply(lambda x: math.ceil(float(x/7)))
studentVle3.loc[studentVle3['date'] >=0, '_date_group'] = studentVle3['_date_group'].apply(lambda x: math.floor(float((x/7)+1)))

# for EEE Course, Cut off filter at Week 16
studentVle3_09 = studentVle3[(studentVle3['_date_group']<=9)]
studentVle3_18 = studentVle3[(studentVle3['_date_group']<=18)]
studentVle3_23 = studentVle3[(studentVle3['_date_group']<=23)]
studentVle3_30 = studentVle3[(studentVle3['_date_group']<=30)]

In [7]:
# sum up
tbl_sum_click_total_09=studentVle3_09.\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_total_09.columns = ['sum_click_total']
tbl_sum_click_total_09 = tbl_sum_click_total_09.reset_index()

tbl_sum_click_total_18=studentVle3_18.\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_total_18.columns = ['sum_click_total']
tbl_sum_click_total_18 = tbl_sum_click_total_18.reset_index()

tbl_sum_click_total_23=studentVle3_23.\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_total_23.columns = ['sum_click_total']
tbl_sum_click_total_23 = tbl_sum_click_total_23.reset_index()

tbl_sum_click_total_30=studentVle3_30.\
            set_index(['id_student','code_module','code_presentation']).\
            groupby(['id_student','code_module','code_presentation']).\
            agg({'sum_click':['sum']})
tbl_sum_click_total_30.columns = ['sum_click_total']
tbl_sum_click_total_30 = tbl_sum_click_total_30.reset_index()
##################################
# create distinct list
studentVle3_distinct=studentVle3[['id_student','code_module','code_presentation']]
studentVle3_distinct=studentVle3_distinct.drop_duplicates()

##################################
# merge
studentVle3_agg1_09 = studentVle3_distinct\
    .merge(tbl_sum_click_total_09, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) 

studentVle3_agg1_18 = studentVle3_distinct\
    .merge(tbl_sum_click_total_18, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) 

studentVle3_agg1_23 = studentVle3_distinct\
    .merge(tbl_sum_click_total_23, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"])

studentVle3_agg1_30 = studentVle3_distinct\
    .merge(tbl_sum_click_total_30, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"])

In [8]:
DatasetForModel_TableOne_09 = studentInfo3\
    .merge(studentAssessment2_agg1_09, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) \
    .merge(studentVle3_agg1_09, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) 

DatasetForModel_TableOne_18 = studentInfo3\
    .merge(studentAssessment2_agg1_18, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) \
    .merge(studentVle3_agg1_18, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) 

DatasetForModel_TableOne_23 = studentInfo3\
    .merge(studentAssessment2_agg1_23, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) \
    .merge(studentVle3_agg1_23, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) 

DatasetForModel_TableOne_30 = studentInfo3\
    .merge(studentAssessment2_agg1_30, how='left', \
           left_on=["id_student","code_module", "code_presentation"], \
           right_on=["id_student","code_module", "code_presentation"]) \
    .merge(studentVle3_agg1_30, how='left', \
         left_on=["id_student","code_module", "code_presentation"], \
         right_on=["id_student","code_module", "code_presentation"]) 

In [9]:
print(9*7)
print(18*7)
print(23*7)
print(30*7)

63
126
161
210


In [10]:
DatasetForModel_TableOne_09.loc[DatasetForModel_TableOne_09['date_unregistration'].isnull(), '_flag_dropout_by_Week09'] = 0
DatasetForModel_TableOne_09.loc[DatasetForModel_TableOne_09['date_unregistration']>63, '_flag_dropout_by_Week09'] = 0
DatasetForModel_TableOne_09.loc[DatasetForModel_TableOne_09['date_unregistration']<=63, '_flag_dropout_by_Week09'] = 1
DatasetForModel_TableOne_18.loc[DatasetForModel_TableOne_18['date_unregistration'].isnull(), '_flag_dropout_by_Week18'] = 0
DatasetForModel_TableOne_18.loc[DatasetForModel_TableOne_18['date_unregistration']>126, '_flag_dropout_by_Week18'] = 0
DatasetForModel_TableOne_18.loc[DatasetForModel_TableOne_18['date_unregistration']<=126, '_flag_dropout_by_Week18'] = 1
DatasetForModel_TableOne_23.loc[DatasetForModel_TableOne_23['date_unregistration'].isnull(), '_flag_dropout_by_Week23'] = 0
DatasetForModel_TableOne_23.loc[DatasetForModel_TableOne_23['date_unregistration']>161, '_flag_dropout_by_Week23'] = 0
DatasetForModel_TableOne_23.loc[DatasetForModel_TableOne_23['date_unregistration']<=161, '_flag_dropout_by_Week23'] = 1
DatasetForModel_TableOne_30.loc[DatasetForModel_TableOne_30['date_unregistration'].isnull(), '_flag_dropout_by_Week30'] = 0
DatasetForModel_TableOne_30.loc[DatasetForModel_TableOne_30['date_unregistration']>210, '_flag_dropout_by_Week30'] = 0
DatasetForModel_TableOne_30.loc[DatasetForModel_TableOne_30['date_unregistration']<=210, '_flag_dropout_by_Week30'] = 1

In [11]:
DatasetForModel_TableOne_09['_flag_final_result'] = pd.Series([], dtype=int)
DatasetForModel_TableOne_09.loc[(DatasetForModel_TableOne_09['final_result'].isin(['Withdrawn']))|(DatasetForModel_TableOne_09['final_result'].isin(['Fail'])), '_flag_final_result'] = 0
DatasetForModel_TableOne_09.loc[(DatasetForModel_TableOne_09['final_result'].isin(['Distinction']))|(DatasetForModel_TableOne_09['final_result'].isin(['Pass'])), '_flag_final_result'] = 1

DatasetForModel_TableOne_18['_flag_final_result'] = pd.Series([], dtype=int)
DatasetForModel_TableOne_18.loc[(DatasetForModel_TableOne_18['final_result'].isin(['Withdrawn']))|(DatasetForModel_TableOne_18['final_result'].isin(['Fail'])), '_flag_final_result'] = 0
DatasetForModel_TableOne_18.loc[(DatasetForModel_TableOne_18['final_result'].isin(['Distinction']))|(DatasetForModel_TableOne_18['final_result'].isin(['Pass'])), '_flag_final_result'] = 1

DatasetForModel_TableOne_23['_flag_final_result'] = pd.Series([], dtype=int)
DatasetForModel_TableOne_23.loc[(DatasetForModel_TableOne_23['final_result'].isin(['Withdrawn']))|(DatasetForModel_TableOne_23['final_result'].isin(['Fail'])), '_flag_final_result'] = 0
DatasetForModel_TableOne_23.loc[(DatasetForModel_TableOne_23['final_result'].isin(['Distinction']))|(DatasetForModel_TableOne_23['final_result'].isin(['Pass'])), '_flag_final_result'] = 1

DatasetForModel_TableOne_30['_flag_final_result'] = pd.Series([], dtype=int)
DatasetForModel_TableOne_30.loc[(DatasetForModel_TableOne_30['final_result'].isin(['Withdrawn']))|(DatasetForModel_TableOne_30['final_result'].isin(['Fail'])), '_flag_final_result'] = 0
DatasetForModel_TableOne_30.loc[(DatasetForModel_TableOne_30['final_result'].isin(['Distinction']))|(DatasetForModel_TableOne_30['final_result'].isin(['Pass'])), '_flag_final_result'] = 1

In [12]:
DatasetForModel_TableOne_09_EEE = DatasetForModel_TableOne_09[DatasetForModel_TableOne_09['code_module'].isin(['EEE'])]
DatasetForModel_TableOne_18_EEE = DatasetForModel_TableOne_18[DatasetForModel_TableOne_18['code_module'].isin(['EEE'])]
DatasetForModel_TableOne_23_EEE = DatasetForModel_TableOne_23[DatasetForModel_TableOne_23['code_module'].isin(['EEE'])]
DatasetForModel_TableOne_30_EEE = DatasetForModel_TableOne_30[DatasetForModel_TableOne_30['code_module'].isin(['EEE'])]

DatasetForModel_TableOne_09_EEE['final_weighted_score'] = DatasetForModel_TableOne_09_EEE['sum_weighted_score']/100
DatasetForModel_TableOne_18_EEE['final_weighted_score'] = DatasetForModel_TableOne_18_EEE['sum_weighted_score']/100
DatasetForModel_TableOne_23_EEE['final_weighted_score'] = DatasetForModel_TableOne_23_EEE['sum_weighted_score']/100
DatasetForModel_TableOne_30_EEE['final_weighted_score'] = DatasetForModel_TableOne_30_EEE['sum_weighted_score']/100

<ipython-input-12-c0910b170c07>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DatasetForModel_TableOne_09_EEE['final_weighted_score'] = DatasetForModel_TableOne_09_EEE['sum_weighted_score']/100
<ipython-input-12-c0910b170c07>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DatasetForModel_TableOne_18_EEE['final_weighted_score'] = DatasetForModel_TableOne_18_EEE['sum_weighted_score']/100
<ipython-input-12-c0910b170c07>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [13]:
EEE_Week09 = DatasetForModel_TableOne_09_EEE[DatasetForModel_TableOne_09_EEE['_flag_dropout_by_Week09'].isin([0])]
EEE_Week18 = DatasetForModel_TableOne_18_EEE[DatasetForModel_TableOne_18_EEE['_flag_dropout_by_Week18'].isin([0])]
EEE_Week23 = DatasetForModel_TableOne_23_EEE[DatasetForModel_TableOne_23_EEE['_flag_dropout_by_Week23'].isin([0])]
EEE_Week30 = DatasetForModel_TableOne_30_EEE[DatasetForModel_TableOne_30_EEE['_flag_dropout_by_Week30'].isin([0])]

EEE_Week09.to_csv('EEE_Week09.csv',index=True,header=True)
EEE_Week18.to_csv('EEE_Week18.csv',index=True,header=True)
EEE_Week23.to_csv('EEE_Week23.csv',index=True,header=True)
EEE_Week30.to_csv('EEE_Week30.csv',index=True,header=True)

#######################################

### Optimize Model, Data Wise

In [1]:
import os
import pandas as pd
import numpy as np
os.getcwd()
os.chdir('C:\\Users\\EeLian\\Desktop\\OULAD_KAGGLE\\PythonCodeCleanUp\\Data')
EEE_Week09 = pd.read_csv('EEE_Week09.csv')
EEE_Week18 = pd.read_csv('EEE_Week18.csv')
EEE_Week23 = pd.read_csv('EEE_Week23.csv')
EEE_Week30 = pd.read_csv('EEE_Week30.csv')

In [2]:
# update as boolean
EEE_Week09['_flag_final_result'] = EEE_Week09['_flag_final_result'].astype(bool)
EEE_Week18['_flag_final_result'] = EEE_Week18['_flag_final_result'].astype(bool)
EEE_Week23['_flag_final_result'] = EEE_Week23['_flag_final_result'].astype(bool)
EEE_Week30['_flag_final_result'] = EEE_Week30['_flag_final_result'].astype(bool)

In [3]:
# create surrogate keys
EEE_Week09['_surrogate_key_four']=\
    EEE_Week09['code_module']+ \
    EEE_Week09['code_presentation']
EEE_Week09['_surrogate_key_three']=\
    EEE_Week09['id_student'].astype(str) + \
    EEE_Week09['code_module']+ \
    EEE_Week09['code_presentation']

EEE_Week18['_surrogate_key_four']=\
    EEE_Week18['code_module']+ \
    EEE_Week18['code_presentation']
EEE_Week18['_surrogate_key_three']=\
    EEE_Week18['id_student'].astype(str) + \
    EEE_Week18['code_module']+ \
    EEE_Week18['code_presentation']

EEE_Week23['_surrogate_key_four']=\
    EEE_Week23['code_module']+ \
    EEE_Week23['code_presentation']
EEE_Week23['_surrogate_key_three']=\
    EEE_Week23['id_student'].astype(str) + \
    EEE_Week23['code_module']+ \
    EEE_Week23['code_presentation']

EEE_Week30['_surrogate_key_four']=\
    EEE_Week30['code_module']+ \
    EEE_Week30['code_presentation']
EEE_Week30['_surrogate_key_three']=\
    EEE_Week30['id_student'].astype(str) + \
    EEE_Week30['code_module']+ \
    EEE_Week30['code_presentation']

##### Q40

In [4]:
# filter only for EEE2014J
EEE2014J_Week09=EEE_Week09[EEE_Week09['_surrogate_key_four'].isin(['EEE2014J'])]
EEE2014J_Week09_Q=EEE2014J_Week09[['_surrogate_key_three','_flag_final_result','final_weighted_score']]
EEE2014J_Week09_Q=EEE2014J_Week09_Q.fillna(0)
EEE2013J2014B_Week09=EEE_Week09[EEE_Week09['_surrogate_key_four'].isin(['EEE2013J','EEE2014B'])]
EEE2013J2014B_Week09_Q=EEE2013J2014B_Week09[['_surrogate_key_three','_flag_final_result','final_weighted_score']]
EEE2013J2014B_Week09_Q=EEE2013J2014B_Week09_Q.fillna(0)

EEE2014J_Week18=EEE_Week18[EEE_Week18['_surrogate_key_four'].isin(['EEE2014J'])]
EEE2014J_Week18_Q=EEE2014J_Week18[['_surrogate_key_three','_flag_final_result','final_weighted_score']]
EEE2014J_Week18_Q=EEE2014J_Week18_Q.fillna(0)
EEE2013J2014B_Week18=EEE_Week18[EEE_Week18['_surrogate_key_four'].isin(['EEE2013J','EEE2014B'])]
EEE2013J2014B_Week18_Q=EEE2013J2014B_Week18[['_surrogate_key_three','_flag_final_result','final_weighted_score']]
EEE2013J2014B_Week18_Q=EEE2013J2014B_Week18_Q.fillna(0)

EEE2014J_Week23=EEE_Week23[EEE_Week23['_surrogate_key_four'].isin(['EEE2014J'])]
EEE2014J_Week23_Q=EEE2014J_Week23[['_surrogate_key_three','_flag_final_result','final_weighted_score']]
EEE2014J_Week23_Q=EEE2014J_Week23_Q.fillna(0)
EEE2013J2014B_Week23=EEE_Week23[EEE_Week23['_surrogate_key_four'].isin(['EEE2013J','EEE2014B'])]
EEE2013J2014B_Week23_Q=EEE2013J2014B_Week23[['_surrogate_key_three','_flag_final_result','final_weighted_score']]
EEE2013J2014B_Week23_Q=EEE2013J2014B_Week23_Q.fillna(0)

EEE2014J_Week30=EEE_Week30[EEE_Week30['_surrogate_key_four'].isin(['EEE2014J'])]
EEE2014J_Week30_Q=EEE2014J_Week30[['_surrogate_key_three','_flag_final_result','final_weighted_score']]
EEE2014J_Week30_Q=EEE2014J_Week30_Q.fillna(0)
EEE2013J2014B_Week30=EEE_Week30[EEE_Week30['_surrogate_key_four'].isin(['EEE2013J','EEE2014B'])]
EEE2013J2014B_Week30_Q=EEE2013J2014B_Week30[['_surrogate_key_three','_flag_final_result','final_weighted_score']]
EEE2013J2014B_Week30_Q=EEE2013J2014B_Week30_Q.fillna(0)

# find quantile
print(EEE2013J2014B_Week09_Q['final_weighted_score'].quantile(0.4))
print(EEE2013J2014B_Week18_Q['final_weighted_score'].quantile(0.4))
print(EEE2013J2014B_Week23_Q['final_weighted_score'].quantile(0.4))
print(EEE2013J2014B_Week30_Q['final_weighted_score'].quantile(0.4))
print(EEE2014J_Week09_Q['final_weighted_score'].quantile(0.4))
print(EEE2014J_Week18_Q['final_weighted_score'].quantile(0.4))
print(EEE2014J_Week23_Q['final_weighted_score'].quantile(0.4))
print(EEE2014J_Week30_Q['final_weighted_score'].quantile(0.4))

EEE2014J_Week09_Q40=EEE2014J_Week09_Q
EEE2014J_Week09_Q40.loc[EEE2014J_Week09_Q40['final_weighted_score'] <(EEE2014J_Week09_Q['final_weighted_score'].quantile(0.4)), '_bin_predicted_results_baseline'] = False
EEE2014J_Week09_Q40.loc[EEE2014J_Week09_Q40['final_weighted_score'] >=(EEE2014J_Week09_Q['final_weighted_score'].quantile(0.4)), '_bin_predicted_results_baseline'] = True

EEE2014J_Week18_Q40=EEE2014J_Week18_Q
EEE2014J_Week18_Q40.loc[EEE2014J_Week18_Q40['final_weighted_score'] <(EEE2014J_Week18_Q['final_weighted_score'].quantile(0.4)), '_bin_predicted_results_baseline'] = False
EEE2014J_Week18_Q40.loc[EEE2014J_Week18_Q40['final_weighted_score'] >=(EEE2014J_Week18_Q['final_weighted_score'].quantile(0.4)), '_bin_predicted_results_baseline'] = True

EEE2014J_Week23_Q40=EEE2014J_Week23_Q
EEE2014J_Week23_Q40.loc[EEE2014J_Week23_Q40['final_weighted_score'] <(EEE2014J_Week23_Q['final_weighted_score'].quantile(0.4)), '_bin_predicted_results_baseline'] = False
EEE2014J_Week23_Q40.loc[EEE2014J_Week23_Q40['final_weighted_score'] >=(EEE2014J_Week23_Q['final_weighted_score'].quantile(0.4)), '_bin_predicted_results_baseline'] = True

EEE2014J_Week30_Q40=EEE2014J_Week30_Q
EEE2014J_Week30_Q40.loc[EEE2014J_Week30_Q40['final_weighted_score'] <(EEE2014J_Week30_Q['final_weighted_score'].quantile(0.4)), '_bin_predicted_results_baseline'] = False
EEE2014J_Week30_Q40.loc[EEE2014J_Week30_Q40['final_weighted_score'] >=(EEE2014J_Week30_Q['final_weighted_score'].quantile(0.4)), '_bin_predicted_results_baseline'] = True

# rename columns
EEE2014J_Week09_Q40.rename(columns = {'_flag_final_result':'_bin_actual_final_result', 'final_weighted_score':'final_weighted_score_wk09','_bin_predicted_results_baseline':'_bin_predicted_results_baseline_wk09'}, inplace = True)
EEE2014J_Week18_Q40.rename(columns = {'_flag_final_result':'_bin_actual_final_result', 'final_weighted_score':'final_weighted_score_wk18','_bin_predicted_results_baseline':'_bin_predicted_results_baseline_wk18'}, inplace = True)
EEE2014J_Week23_Q40.rename(columns = {'_flag_final_result':'_bin_actual_final_result', 'final_weighted_score':'final_weighted_score_wk23','_bin_predicted_results_baseline':'_bin_predicted_results_baseline_wk23'}, inplace = True)
EEE2014J_Week30_Q40.rename(columns = {'_flag_final_result':'_bin_actual_final_result', 'final_weighted_score':'final_weighted_score_wk30','_bin_predicted_results_baseline':'_bin_predicted_results_baseline_wk30'}, inplace = True)

# cross tabs
actual_vs_baselinepred_wk09_Q40=pd.crosstab(EEE2014J_Week09_Q40['_bin_actual_final_result'], EEE2014J_Week09_Q40['_bin_predicted_results_baseline_wk09'])
actual_vs_baselinepred_wk18_Q40=pd.crosstab(EEE2014J_Week18_Q40['_bin_actual_final_result'], EEE2014J_Week18_Q40['_bin_predicted_results_baseline_wk18'])
actual_vs_baselinepred_wk23_Q40=pd.crosstab(EEE2014J_Week23_Q40['_bin_actual_final_result'], EEE2014J_Week23_Q40['_bin_predicted_results_baseline_wk23'])
actual_vs_baselinepred_wk30_Q40=pd.crosstab(EEE2014J_Week30_Q40['_bin_actual_final_result'], EEE2014J_Week30_Q40['_bin_predicted_results_baseline_wk30'])


13.44
49.04
50.184000000000005
51.04
13.76
37.248000000000005
37.488
37.656


In [5]:
### Period of Exams varies for EEE2014B & EEE2013J
### Hence we use only EEE2014J

In [6]:
print(actual_vs_baselinepred_wk09_Q40)
print("")
print(actual_vs_baselinepred_wk18_Q40)
print("")
print(actual_vs_baselinepred_wk23_Q40)
print("")
print(actual_vs_baselinepred_wk30_Q40)

_bin_predicted_results_baseline_wk09  False  True 
_bin_actual_final_result                          
False                                   208    101
True                                    169    515

_bin_predicted_results_baseline_wk18  False  True 
_bin_actual_final_result                          
False                                   196     59
True                                    180    504

_bin_predicted_results_baseline_wk23  False  True 
_bin_actual_final_result                          
False                                   176     49
True                                    188    496

_bin_predicted_results_baseline_wk30  False  True 
_bin_actual_final_result                          
False                                   164     44
True                                    193    491


In [7]:
print("actual_vs_baselinepred40_wk09")
print("Specificity: ", (208)/(101+208))
print("Accuracy: ", (208+515)/(208+101+169+515))
print("Recall: ",(515/(169+515)))
print("Precision: ",(515/(101+515)))
print("F1 Score: ",(2*(515/(169+515))*(515/(101+515)))/((515/(169+515))+(515/(101+515))))
print("AUC: ",0.5-((101/(101+208))/2)+((515/(169+515))/2))
print("")
print("actual_vs_baselinepred40_wk18")
print("Specificity: ", ((196)/(196+59)))
print("Accuracy: ", ((196+504)/(196+59+180+504)))
print("Recall: ",(504/(180+504)))
print("Precision: ",(504/(59+504)))
print("F1 Score: ",(2*(504/(59+504))*(504/(180+504)))/((504/(59+504))+(504/(180+504))))
print("AUC: ",0.5-((59/(59+196))/2)+((504/(180+504))/2))
print("")
print("actual_vs_baselinepred40_wk23")
print("Specificity: ", (176)/(49+176))
print("Accuracy: ", (176+496)/(176+49+188+496))
print("Recall: ",(496/(188+496)))
print("Precision: ",(496/(49+496)))
print("F1 Score: ",(2*(496/(188+496))*(496/(49+496)))/((496/(188+496))+(496/(49+496))))
print("AUC: ",0.5-((49/(49+176))/2)+((496/(188+496))/2))
print("")
print("actual_vs_baselinepred40_wk30")
print("Specificity: ", (164)/(44+164))
print("Accuracy: ", (164+491)/(164+44+193+491))
print("Recall: ",(491/(193+491)))
print("Precision: ",(491/(44+491)))
print("F1 Score: ",(2*(491/(193+491))*(491/(44+491)))/((491/(193+491))+(491/(44+491))))
print("AUC: ",0.5-((44/(44+164))/2)+((491/(193+491))/2))

actual_vs_baselinepred40_wk09
Specificity:  0.6731391585760518
Accuracy:  0.7280966767371602
Recall:  0.7529239766081871
Precision:  0.836038961038961
F1 Score:  0.7923076923076923
AUC:  0.7130315675921195

actual_vs_baselinepred40_wk18
Specificity:  0.7686274509803922
Accuracy:  0.7454739084132055
Recall:  0.7368421052631579
Precision:  0.8952042628774423
F1 Score:  0.8083400160384924
AUC:  0.752734778121775

actual_vs_baselinepred40_wk23
Specificity:  0.7822222222222223
Accuracy:  0.7392739273927392
Recall:  0.7251461988304093
Precision:  0.9100917431192661
F1 Score:  0.807160292921074
AUC:  0.7536842105263157

actual_vs_baselinepred40_wk30
Specificity:  0.7884615384615384
Accuracy:  0.734304932735426
Recall:  0.7178362573099415
Precision:  0.9177570093457944
F1 Score:  0.8055783429040198
AUC:  0.7531488978857399


In [8]:
EEE2014J_Week09_Q40=EEE2014J_Week09_Q40.replace({
    '_bin_actual_final_result': {
        True: 'ActualPass',
        False:'ActualFail'
    }
    ,'_bin_predicted_results_baseline_wk09': {
        True: 'PredictedPass',
        False:'PredictedFail'
    }
})

EEE2014J_Week18_Q40=EEE2014J_Week18_Q40.replace({
    '_bin_actual_final_result': {
        True: 'ActualPass',
        False:'ActualFail'
    }
    ,'_bin_predicted_results_baseline_wk18': {
        True: 'PredictedPass',
        False:'PredictedFail'
    }
})

EEE2014J_Week23_Q40=EEE2014J_Week23_Q40.replace({
    '_bin_actual_final_result': {
        True: 'ActualPass',
        False:'ActualFail'
    }
    ,'_bin_predicted_results_baseline_wk23': {
        True: 'PredictedPass',
        False:'PredictedFail'
    }
})

EEE2014J_Week30_Q40=EEE2014J_Week30_Q40.replace({
    '_bin_actual_final_result': {
        True: 'ActualPass',
        False:'ActualFail'
    }
    ,'_bin_predicted_results_baseline_wk30': {
        True: 'PredictedPass',
        False:'PredictedFail'
    }
})

In [9]:
# export to csv
EEE2014J_Week09_Q40.to_csv('Dataset_Dashboard_Table1_Predictions_Week09_Q40.csv',index=True,header=True)
EEE2014J_Week18_Q40.to_csv('Dataset_Dashboard_Table1_Predictions_Week18_Q40.csv',index=True,header=True)
EEE2014J_Week23_Q40.to_csv('Dataset_Dashboard_Table1_Predictions_Week23_Q40.csv',index=True,header=True)
EEE2014J_Week30_Q40.to_csv('Dataset_Dashboard_Table1_Predictions_Week30_Q40.csv',index=True,header=True)

THE END

##### Q37

In [10]:
# filter only for EEE2014J
EEE2014J_Week18=EEE_Week18[EEE_Week18['_surrogate_key_four'].isin(['EEE2014J'])]
EEE2014J_Week18_Q=EEE2014J_Week18[['_surrogate_key_three','_flag_final_result','final_weighted_score']]
EEE2014J_Week18_Q=EEE2014J_Week18_Q.fillna(0)

EEE2014J_Week23=EEE_Week23[EEE_Week23['_surrogate_key_four'].isin(['EEE2014J'])]
EEE2014J_Week23_Q=EEE2014J_Week23[['_surrogate_key_three','_flag_final_result','final_weighted_score']]
EEE2014J_Week23_Q=EEE2014J_Week23_Q.fillna(0)

EEE2014J_Week30=EEE_Week30[EEE_Week30['_surrogate_key_four'].isin(['EEE2014J'])]
EEE2014J_Week30_Q=EEE2014J_Week30[['_surrogate_key_three','_flag_final_result','final_weighted_score']]
EEE2014J_Week30_Q=EEE2014J_Week30_Q.fillna(0)

# find quantile
print(EEE2014J_Week18_Q['final_weighted_score'].quantile(0.37))
print(EEE2014J_Week23_Q['final_weighted_score'].quantile(0.37))
print(EEE2014J_Week30_Q['final_weighted_score'].quantile(0.37))

EEE2014J_Week18_Q37=EEE2014J_Week18_Q
EEE2014J_Week18_Q37.loc[EEE2014J_Week18_Q37['final_weighted_score'] <(EEE2014J_Week18_Q['final_weighted_score'].quantile(0.37)), '_bin_predicted_results_baseline'] = False
EEE2014J_Week18_Q37.loc[EEE2014J_Week18_Q37['final_weighted_score'] >=(EEE2014J_Week18_Q['final_weighted_score'].quantile(0.37)), '_bin_predicted_results_baseline'] = True

EEE2014J_Week23_Q37=EEE2014J_Week23_Q
EEE2014J_Week23_Q37.loc[EEE2014J_Week23_Q37['final_weighted_score'] <(EEE2014J_Week23_Q['final_weighted_score'].quantile(0.37)), '_bin_predicted_results_baseline'] = False
EEE2014J_Week23_Q37.loc[EEE2014J_Week23_Q37['final_weighted_score'] >=(EEE2014J_Week23_Q['final_weighted_score'].quantile(0.37)), '_bin_predicted_results_baseline'] = True

EEE2014J_Week30_Q37=EEE2014J_Week30_Q
EEE2014J_Week30_Q37.loc[EEE2014J_Week30_Q37['final_weighted_score'] <(EEE2014J_Week30_Q['final_weighted_score'].quantile(0.37)), '_bin_predicted_results_baseline'] = False
EEE2014J_Week30_Q37.loc[EEE2014J_Week30_Q37['final_weighted_score'] >=(EEE2014J_Week30_Q['final_weighted_score'].quantile(0.37)), '_bin_predicted_results_baseline'] = True


EEE2014J_Week18_Q37.rename(columns = {'_flag_final_result':'_bin_actual_final_result', 'final_weighted_score':'final_weighted_score_wk18','_bin_predicted_results_baseline':'_bin_predicted_results_baseline_wk18'}, inplace = True)
EEE2014J_Week23_Q37.rename(columns = {'_flag_final_result':'_bin_actual_final_result', 'final_weighted_score':'final_weighted_score_wk23','_bin_predicted_results_baseline':'_bin_predicted_results_baseline_wk23'}, inplace = True)
EEE2014J_Week30_Q37.rename(columns = {'_flag_final_result':'_bin_actual_final_result', 'final_weighted_score':'final_weighted_score_wk30','_bin_predicted_results_baseline':'_bin_predicted_results_baseline_wk30'}, inplace = True)

actual_vs_baselinepred_wk18_Q37=pd.crosstab(EEE2014J_Week18_Q37['_bin_actual_final_result'], EEE2014J_Week18_Q37['_bin_predicted_results_baseline_wk18'])
actual_vs_baselinepred_wk23_Q37=pd.crosstab(EEE2014J_Week23_Q37['_bin_actual_final_result'], EEE2014J_Week23_Q37['_bin_predicted_results_baseline_wk23'])
actual_vs_baselinepred_wk30_Q37=pd.crosstab(EEE2014J_Week30_Q37['_bin_actual_final_result'], EEE2014J_Week30_Q37['_bin_predicted_results_baseline_wk30'])

36.8
36.88
37.0536


In [11]:
print(actual_vs_baselinepred_wk18_Q37)
print("")
print(actual_vs_baselinepred_wk23_Q37)
print("")
print(actual_vs_baselinepred_wk30_Q37)

_bin_predicted_results_baseline_wk18  False  True 
_bin_actual_final_result                          
False                                   189     66
True                                    154    530

_bin_predicted_results_baseline_wk23  False  True 
_bin_actual_final_result                          
False                                   171     54
True                                    162    522

_bin_predicted_results_baseline_wk30  False  True 
_bin_actual_final_result                          
False                                   161     47
True                                    169    515


##### Q35

In [12]:
# filter only for EEE2014J
EEE2014J_Week18=EEE_Week18[EEE_Week18['_surrogate_key_four'].isin(['EEE2014J'])]
EEE2014J_Week18_Q=EEE2014J_Week18[['_surrogate_key_three','_flag_final_result','final_weighted_score']]
EEE2014J_Week18_Q=EEE2014J_Week18_Q.fillna(0)

EEE2014J_Week23=EEE_Week23[EEE_Week23['_surrogate_key_four'].isin(['EEE2014J'])]
EEE2014J_Week23_Q=EEE2014J_Week23[['_surrogate_key_three','_flag_final_result','final_weighted_score']]
EEE2014J_Week23_Q=EEE2014J_Week23_Q.fillna(0)

EEE2014J_Week30=EEE_Week30[EEE_Week30['_surrogate_key_four'].isin(['EEE2014J'])]
EEE2014J_Week30_Q=EEE2014J_Week30[['_surrogate_key_three','_flag_final_result','final_weighted_score']]
EEE2014J_Week30_Q=EEE2014J_Week30_Q.fillna(0)

# find quantile
print(EEE2014J_Week18_Q['final_weighted_score'].quantile(0.35))
print(EEE2014J_Week23_Q['final_weighted_score'].quantile(0.35))
print(EEE2014J_Week30_Q['final_weighted_score'].quantile(0.35))

EEE2014J_Week18_Q35=EEE2014J_Week18_Q
EEE2014J_Week18_Q35.loc[EEE2014J_Week18_Q35['final_weighted_score'] <(EEE2014J_Week18_Q['final_weighted_score'].quantile(0.35)), '_bin_predicted_results_baseline'] = False
EEE2014J_Week18_Q35.loc[EEE2014J_Week18_Q35['final_weighted_score'] >=(EEE2014J_Week18_Q['final_weighted_score'].quantile(0.35)), '_bin_predicted_results_baseline'] = True

EEE2014J_Week23_Q35=EEE2014J_Week23_Q
EEE2014J_Week23_Q35.loc[EEE2014J_Week23_Q35['final_weighted_score'] <(EEE2014J_Week23_Q['final_weighted_score'].quantile(0.35)), '_bin_predicted_results_baseline'] = False
EEE2014J_Week23_Q35.loc[EEE2014J_Week23_Q35['final_weighted_score'] >=(EEE2014J_Week23_Q['final_weighted_score'].quantile(0.35)), '_bin_predicted_results_baseline'] = True

EEE2014J_Week30_Q35=EEE2014J_Week30_Q
EEE2014J_Week30_Q35.loc[EEE2014J_Week30_Q35['final_weighted_score'] <(EEE2014J_Week30_Q['final_weighted_score'].quantile(0.35)), '_bin_predicted_results_baseline'] = False
EEE2014J_Week30_Q35.loc[EEE2014J_Week30_Q35['final_weighted_score'] >=(EEE2014J_Week30_Q['final_weighted_score'].quantile(0.35)), '_bin_predicted_results_baseline'] = True


EEE2014J_Week18_Q35.rename(columns = {'_flag_final_result':'_bin_actual_final_result', 'final_weighted_score':'final_weighted_score_wk18','_bin_predicted_results_baseline':'_bin_predicted_results_baseline_wk18'}, inplace = True)
EEE2014J_Week23_Q35.rename(columns = {'_flag_final_result':'_bin_actual_final_result', 'final_weighted_score':'final_weighted_score_wk23','_bin_predicted_results_baseline':'_bin_predicted_results_baseline_wk23'}, inplace = True)
EEE2014J_Week30_Q35.rename(columns = {'_flag_final_result':'_bin_actual_final_result', 'final_weighted_score':'final_weighted_score_wk30','_bin_predicted_results_baseline':'_bin_predicted_results_baseline_wk30'}, inplace = True)

actual_vs_baselinepred_wk18_Q35=pd.crosstab(EEE2014J_Week18_Q35['_bin_actual_final_result'], EEE2014J_Week18_Q35['_bin_predicted_results_baseline_wk18'])
actual_vs_baselinepred_wk23_Q35=pd.crosstab(EEE2014J_Week23_Q35['_bin_actual_final_result'], EEE2014J_Week23_Q35['_bin_predicted_results_baseline_wk23'])
actual_vs_baselinepred_wk30_Q35=pd.crosstab(EEE2014J_Week30_Q35['_bin_actual_final_result'], EEE2014J_Week30_Q35['_bin_predicted_results_baseline_wk30'])

36.372
36.64
36.794


In [13]:
print(actual_vs_baselinepred_wk18_Q35)
print("")
print(actual_vs_baselinepred_wk23_Q35)
print("")
print(actual_vs_baselinepred_wk30_Q35)

_bin_predicted_results_baseline_wk18  False  True 
_bin_actual_final_result                          
False                                   188     67
True                                    141    543

_bin_predicted_results_baseline_wk23  False  True 
_bin_actual_final_result                          
False                                   169     56
True                                    148    536

_bin_predicted_results_baseline_wk30  False  True 
_bin_actual_final_result                          
False                                   158     50
True                                    154    530
